# AI Agents

In this section, we'll explore what makes an AI system "agentic" and why building these systems reliably is more challenging than simple chains. You'll:

- Understand LLM agency
- Cover a bit of context engineering
- Create some tools
- Build a couple of durable AI agents (non-looping, and looping)

### Demo (Expand for instructor notes or to run on your own)
<!--
1. Clone this repository: `https://github.com/cdavisafc/non-deterministic-ai-agents`. The instructions will also be in the README.
2. Run the Worker: `uv run python -m worker`
3. Start the agent: `uv run python -m start_workflow`
4. Emphasize that this demo showcases the core principle of agentic applications: we give **agency** to the **LLM** to drive the flow of the application. Unlike traditional applications where developers hardcode every decision and branching path, this agent makes its own choices at runtime. The LLM decides which actions to take, when to gather more information, and when the goal has been achieved—all autonomously.
-->

### Let's Build Another Agent!

First, let's set up your notebook. Run the following code blocks to install various packages and tools necessary to run this notebook

### Notebook Setup

Run the following code blocks to install various packages and tools necessary to run this notebook

**Be sure to add your .env file again. It doesn't persist across notebooks or sesions**

In [ ]:
# We'll first install the necessary packages for this workshop.

%pip install --quiet temporalio python-dotenv openai httpx

In [ ]:
# Create .env file
with open(".env", "w") as fh:
  fh.write("OPENAI_API_KEY = YOUR_API_KEY\nLLM_MODEL = openai/gpt-4o")

  # Now open the file and replace YOUR_API_KEY with your API key.

In [ ]:
# Load environment variables and configure LLM settings
import os
from dotenv import load_dotenv

load_dotenv(override=True)

# Get OPENAI_API_KEY environment variable and print it to make sure that your .env file is properly loaded.
LLM_MODEL = os.getenv("LLM_MODEL", "openai/gpt-4o")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", None)
print("OPENAI_API_KEY", OPENAI_API_KEY)

In [ ]:
# This allows us to run the Temporal Asyncio event loop within the event loop of Jupyter Notebooks
import nest_asyncio
nest_asyncio.apply()

In [ ]:
 # Running this will download the Temporal CLI

!curl -sSf https://temporal.download/cli.sh | sh

### Make Sure Your Temporal Web UI is Running

1. You should have the Temporal Server running in your terminal (run `temporal server start-dev` if not).
2. Then in your `Ports` tab on the bottom of this screen, find `8233` and click on the Globe icon to open the Temporal Web UI.

### Building Our Agent: Step by Step

#### Step 1 - Create the Request Dataclass

First, we'll create a dataclass that packages all the parameters needed for our LLM calls. This groups together the model name, agent instructions, conversation input, and available tools in one structure. 

In [ ]:
# TODO: Run this code block to load it into the program
from dataclasses import dataclass
from typing import Any

@dataclass
class OpenAIResponsesRequest:
    model: str
    instructions: str
    input: object
    tools: list[dict[str, Any]]

### Step 2 - Create the Activity for LLM Invocations

We will create an Activity which invokes the OpenAI LLM, taking in our `OpenAIResponsesRequest` dataclass.

It's a reusable function that handles all communication with OpenAI's LLM throughout our agent's execution.


In [ ]:
# Step 1: Supply the `OpenAIResponsesRequest` we just loaded in as the `request` for the parameter
# Step 2: Run this code block to load it into the program
from temporalio import activity
from openai import AsyncOpenAI
from openai.types.responses import Response

@activity.defn
async def create(request: ) -> Response: # TODO: Supply the `OpenAIResponsesRequest` as the `request` for the parameter
    # Temporal best practice: Disable retry logic in OpenAI API client library.
    client = AsyncOpenAI(max_retries=0)

    resp = await client.responses.create(
        model=request.model,
        instructions=request.instructions,
        input=request.input,
        tools=request.tools,
        timeout=30,
    )

    return resp

In [ ]:
# Optional: Run this cell to load and display the solution
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "Solutions_04_AI_Agents" / "create_activity_solution.py"

code = solution_file.read_text()

print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))

### Step 3 - Create the Helper Function

Before we build our tool that fetches weather alerts, we need a helper function that converts Pydantic models into OpenAI's tool definition format.

The `oai_responses_tool_from_model` function accepts a tool name, description, and a Pydantic model, then returns JSON in the format expected by OpenAI's Responses API tool definitions.

In [ ]:
# TODO: Run this code block
from openai.lib._pydantic import to_strict_json_schema
from pydantic import BaseModel

def oai_responses_tool_from_model(name: str, description: str, model: type[BaseModel]):
    return {
        "type": "function",
        "name": name,
        "description": description,
        "parameters": to_strict_json_schema(model),
        "strict": True,
    }

### Step 4 - Set Up the National Weather Service API Wrapper

Before our agent can fetch weather alerts, we need to create helper functions that know how to communicate with the [National Weather Service API](https://www.weather.gov/documentation/services-web-api), specifically for the weather alerts endpoint for a certain state.

These helper functions will be used by our Activity in the next steps.

In [ ]:
# TODO: Run this code block
from typing import Any
import httpx
import json

# Constants for the National Weather Service API
NWS_API_BASE = "https://api.weather.gov"
USER_AGENT = "weather-app/1.0"

def _alerts_url(state: str) -> str:
    """Build the NWS API URL for a given state."""
    return f"{NWS_API_BASE}/alerts/active/area/{state}"

# External calls will happen via activities
async def _make_nws_request(url: str) -> dict[str, Any] | None:
    """Make a request to the NWS API with proper error handling."""
    headers = {
        "User-Agent": USER_AGENT,
        "Accept": "application/geo+json"
    }
    async with httpx.AsyncClient() as client:
        response = await client.get(url, headers=headers, timeout=5.0)
        response.raise_for_status()
        return response.json()

### Step 5 - Build the Tool for the OpenAI Responses API

Now we'll create the Pydantic model that defines what parameters the tool accepts, then use our helper function to convert it into OpenAI's tool definition format. 

We will assign this to our tool registry, `WEATHER_ALERTS_TOOL_OAI`, for later use.

In [ ]:
# Step 1: In the `description` for state, set it to "Two-letter US state code (e.g. CA, NY)"
# Step 2: Run this code block
from pydantic import BaseModel, Field

class GetWeatherAlertsRequest(BaseModel):
    state: str = Field(description="")  # TODO: Set the description to be "Two-letter US state code (e.g. CA, NY)"

WEATHER_ALERTS_TOOL_OAI: dict[str, Any] = oai_responses_tool_from_model(
    "get_weather_alerts", # name 
    "Get weather alerts for a US state.", # description
    GetWeatherAlertsRequest) # model

In [ ]:
# Optional: Run this cell to load and display the solution
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "Solutions_04_AI_Agents" / "GetWeatherAlertsRequest_solution.py"

code = solution_file.read_text()

print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))

### Step 6 - Implement the Activity

We'll create the Activity function that executes when the LLM decides to use this tool. This function calls the NWS API and returns the weather alert data.

In [ ]:
# Step 1: After the `await`, call the `_make_nws_request` Activity you ran earlier.
# Remember: External calls will happen via Activities.
# Step 2: Run this code block
from temporalio import activity

@activity.defn
async def get_weather_alerts(weather_alerts_request: GetWeatherAlertsRequest) -> str:
    """Get weather alerts for a US state.

    Args:
        state: Two-letter US state code (e.g. CA, NY)
    """
    data = await ""(_alerts_url(weather_alerts_request.state)) # TODO: Call the `_make_nws_request` Activity you ran earlier
    return json.dumps(data)

In [ ]:
# Optional: Run this cell to load and display the solution
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "Solutions_04_AI_Agents" / "get_weather_alerts_activity_solution.py"

code = solution_file.read_text()

print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))

### Step 7: Create the Agent

Now we'll create the agent, which is implemented as the **ToolCallingWorkflow**. This Workflow ties everything together:

1. **Makes an initial LLM call** with the user's input and available tools
2. **Checks if the LLM wants to call a tool** by examining the response type
3. **Executes the tool** if requested
4. **Makes a second LLM call** with the tool results to get a final response

In [ ]:
# Step 1: Call the `create` Activity for the initial LLM call
# Step 2: Fill in the `OpenAIResponsesRequest` with `system_instructions`, `input_list`, and `WEATHER_ALERTS_TOOL_OAI`.
# Step 3: Execute the get_weather_alerts activity when a tool is called
# Step 4: Run this code block to load it into the program

from temporalio import workflow
from datetime import timedelta
import json

# sandboxed=False is a Notebook only requirement. You normally don't do this
@workflow.defn(sandboxed=False)
class ToolCallingWorkflow:
    @workflow.run
    async def run(self, input: str) -> str:
        input_list = [{"role": "user", "content": input}]
        
        # Initial LLM call with system instructions and tools
        system_instructions = "if no tools seem to be needed, respond in haikus."
        result = await workflow.execute_activity(
            # TODO: Call the `create` Activity here
            OpenAIResponsesRequest(
                model="gpt-4o-mini",
                instructions="",  # TODO: Fill in with system_instructions
                input="",  # TODO: Fill in with input_list
                tools=[]  # TODO: Fill in tools list with WEATHER_ALERTS_TOOL_OAI
            ),
            start_to_close_timeout=timedelta(seconds=30),
        )
        
        # Process the LLM response
        item = result.output[0]
        
        # if the result is a tool call, call the tool
        if item.type == "function_call":
            if item.name == "get_weather_alerts":

                # serialize the output, which is an OpenAI object
                input_list += [
                    i.model_dump() if hasattr(i, "model_dump") else i
                    for i in result.output
                ]
                
                result = await workflow.execute_activity(
                    "", # TODO: Execute the get_weather_alerts activity
                    GetWeatherAlertsRequest(state=json.loads(item.arguments)["state"]),
                    start_to_close_timeout=timedelta(seconds=30),
                )
                
                # Add tool call result to input list
                input_list.append({
                    "type": "function_call_output",
                    "call_id": item.call_id,
                    "output": result
                })
                
                result = await workflow.execute_activity(
                    create,
                    OpenAIResponsesRequest(
                        model="gpt-4o-mini",
                        instructions="return the tool call result in a readable format",
                        input=input_list,
                        tools=[]
                    ),
                    start_to_close_timeout=timedelta(seconds=30),
                )
        
        result = result.output_text
        return result

In [ ]:
# Optional: Run this cell to load and display the solution
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "Solutions_04_AI_Agents" / "tool_calling_workflow_solution.py"

code = solution_file.read_text()

print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))

### Running the Tool-Calling Agent

Now let's run our agent by running our Worker!

In [ ]:
# Step 1: Set the Task Queue to be "tool-calling-python-task-queue"
# Step 2: Register `ToolCallingWorkflow` on this Worker
# Step 3: Run this code
import concurrent.futures
from temporalio.client import Client
from temporalio.worker import Worker
from temporalio.contrib.pydantic import pydantic_data_converter

async def run_worker() -> None:
    # Create client connected to server at the given address
    client = await Client.connect("localhost:7233", data_converter=pydantic_data_converter)

    # Run the Worker
    with concurrent.futures.ThreadPoolExecutor(max_workers=100) as activity_executor:
        worker = Worker(
            client,
            task_queue="", # TODO Set the Task Queue to be "tool-calling-python-task-queue"
            workflows=[], # TODO Register `ToolCallingWorkflow` on this Worker
            activities=[create, get_weather_alerts],
            activity_executor=activity_executor
        )

        print(f"Starting the worker....")
        await worker.run()

In [ ]:
# Optional: Run this cell to load and display the solution
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "Solutions_04_AI_Agents" / "worker_solution.py"

code = solution_file.read_text()

print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))

In [ ]:
# Run the Worker
import asyncio

worker = asyncio.create_task(run_worker())

### Initiate an Interaction with the Agent

In order to interact with this simple AI agent, we create a Temporal client and execute a workflow.

In [ ]:
# Step 1: Set the task queue of the client to match the task queue the worker is polling
# Step 2: Run this code block
import sys
import uuid

from temporalio.client import Client
from temporalio.contrib.pydantic import pydantic_data_converter

async def main():
    client = await Client.connect(
        "localhost:7233",
        data_converter=pydantic_data_converter,
    )

    query = sys.argv[1] if len(sys.argv) > 1 else "Hello, how are you?"

    # Submit the Tool Calling workflow for execution
    result = await client.execute_workflow(
        ToolCallingWorkflow,
        query,
        id=f"weather-alert-agent-{uuid.uuid4()}",
        task_queue="", # TODO: Set the task queue of the client to match the task queue the worker is polling
    )
    print(f"Result: {result}")

In [ ]:
# Optional: Run this cell to load and display the solution
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "Solutions_04_AI_Agents" / "client_solution.py"

code = solution_file.read_text()

print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))

### Test the Agent

Now let's test the agent with different queries. Run the cells below to see how the agent responds to different types of requests.

In [ ]:
# Test with California weather
import sys
sys.argv = ["", "Are there any weather alerts in CA?"]
await main()

In [ ]:
# Test with New York weather
import sys
sys.argv = ["", "Are there any weather alerts in NY?"]
await main()

In [ ]:
# Test without tools (should get haiku)
import sys
sys.argv = ["", "Tell me a joke"]
await main()

## Observe Your Web UI

Refresh your Web UI. 

- What Activities are called for a weather alert Workflow Execution vs. the joke Workflow Execution?
- Click on the first create Activity in the weather alert Workflow. What was passed in the input field?
- Click on the second create Activity in the weather alert Workflow.  How has the input field changed?

A point to make is that the **LLM decides autonomously** whether to use the weather
alerts tool or not. You don't hardcode "if user mentions a state, call the API" -
instead, the LLM evaluates the input and decides if calling `get_weather_alerts`
will help achieve the goal.

# Thank you for attending this workshop. Feedback?

Please leave your feedback for this workshop [here](https://docs.google.com/forms/d/e/1FAIpQLSfkHMev6KCNGFHpVNydyjgAh2ALeHNVYv9TaSrAoBsT0KmNHQ/viewform?usp=header)!

### What's Next? 

Continue your learning and check out these resources!

### Resources

- [Temporal Docs](https://docs.temporal.io/)
- [Free Temporal Courses](https://learn.temporal.io/courses/)
- Learn more about [AI Agents with Temporal](https://temporal.io/code-exchange/ai-agent-execution-using-temporal)
- Join our [Community Slack](https://t.mp/slack)